In [ ]:
from bs4 import BeautifulSoup
import os
import numpy as np
from tqdm.notebook import tqdm
import re
import random
import string
import pytesseract
from PIL import Image
import imgkit
import pandas as pd
import requests

In [ ]:
DEBUG = False

## IMPORT DATASET


In [ ]:
from datasets import load_dataset
import os
import numpy as np

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Grocery_and_Gourmet_Food", streaming=True, trust_remote_code=True)
print(type(dataset))
splits = dataset.keys()
#print all splits
print(splits)

for example in dataset["full"]:
    print(example)
    break  # Remove this line to process the entire dataset

dataset_iterator = iter(dataset["full"])
#get example[title]
print(next(dataset_iterator)["title"])
#get length of dataset



In [ ]:
import os
import shutil
pwd = os.path.dirname(os.getcwd())
print(pwd)
font_path = os.path.join(pwd, 'DejaVuSans.ttf')
dataset_path = os.path.join(pwd, "data" ,"dataset", "webpages")
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)
else:
    print("Folder already exists, deleting and creating new one")
    shutil.rmtree(dataset_path)
    os.makedirs(dataset_path)




### Function to create custom html file

In [ ]:
import os
def create_html_with_roboto(file_name, title, body, ):
    """
    Create an HTML file with the given title and body, using the Roboto font.

    Parameters:
        title (str): The title of the HTML document.
        body (str): The body content of the HTML document.
        file_name (str): The name of the file to save the content to.
    """
    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>{title}</title>
        <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
        <style>
            body {{
                font-family: 'Roboto', sans-serif;
            }}
        </style>
    </head>
    <body>
        <h1>{title}</h1>
        <p>{body}</p>
    </body>
    </html>
    """
    save_html_to_file(html_content, file_name)

def create_html_with_font(path_file
                          ,title, body,font_path= os.path.join(os.getcwd(), "DejaVuSans.ttf")):
    """
    Create an HTML file with the given title and body, using a specified font.

    Parameters:
        title (str): The title of the HTML document.
        body (str): The body content of the HTML document.
        file_name (str): The name of the file to save the content to.
    """
    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>{title}</title>
        <style>
            @font-face {{
                font-family: 'DejaVuSans';
                src: url('{font_path}');
            }}
            body {{
                font-family: 'DejaVuSans', sans-serif;
            }}
        </style>
    </head>
    <body>
        <h1>{title}</h1>
        <p>{body}</p>
    </body>
    </html>
    """
    save_html_to_file(html_content, path_file)

def save_html_to_file(html_content, path_file):
    """
    Save the modified HTML content to a file.

    Parameters:
        html_content (str): The HTML content as a string.
        file_name (str): The name of the file to save the content to.
    """
    with open(path_file, "w", encoding="utf-8") as file:
        file.write(html_content)
    return path_file



In [ ]:
#test
if DEBUG:
    create_html_with_font("Sample Title", "This is the body content.", os.path.join(dataset_path, "output.html"))
    

In [ ]:
import re
import random
import math

def select_target_word(text):
    words = re.split(r'\W+', text)  # Split by any non-word characters
    words = [word for word in words if word.isalpha()]  # Keep only words with alphabetic characters
    
    if not words:
        raise ValueError("No valid candidate words found.")
    
    valid_words = [word for word in words if len(word) >= 2]  # Filter words by minimum length of 2 characters
    
    if not valid_words:
        raise ValueError("No valid candidate words found with the required length.")
    
    target_word = random.choice(valid_words)
    return target_word

## Zero-Width Characters injection

In [ ]:
class ZeroWidthSPaceAttack:
    '''define list of malicious characters'''
    def __init__(self):
        #define the list of malicious symbols
        self.symbols = [u'\u200b', u'\u200c', u'\u200d', u'\u200e', u'\u200f', #U+200x
                      u'\u202a', u'\u202b', u'\u202c', u'\u202d', #U+202x
                      u'\u2060', u'\u2061', u'\u2062', u'\u2063', u'\u2064',
                    #   u'\u2065', u'\u2066', u'\u2067', u'\u2068', u'\u2069', NOT SUPPORTED BY FONT
                      u'\u206a', u'\u206b', u'\u206c', u'\u206d', u'\u206e' #U+206x
                      ]
        self.num_malicius_chars = len(self.symbols)

    '''insertion of malicious input in the middle of a given word;
        for example, given the word "love", the result is "loXve".
    '''
    def mask1(self, word, index = 0, random = True):
        '''word = target word \n index = index of malicious symbols from the
        given list\nrandom = if random True, a randomic index is selected\n'''

        #check the index
        if index < 0 or index > len(self.symbols):
            raise Exception("Invalid index.")

        #sample the index, if required
        if random:
            index = np.random.randint(len(self.symbols))

        #get the target character
        code = self.symbols[index]

        #prepare the result. it must to be unicode
        poison = []

        #calculate the middle of the word
        mid = len(word) // 2

        #create the final message
        poison.append(word[:mid])
        poison.append(code)
        poison.append(word[mid:])

        poison = ''.join(poison)

        return poison

    '''insertion of malicious input between each character of the word;

        for example, given the word "love", the result is "lXoXvXe".
    '''
    def mask2(self, word, index = 0, random = True):
        '''word = target word\nindex = index of malicious symbols from the
        given list\nrandom = if random True, a randomic index is selected\n'''

        #check the index
        if index < 0 or index > len(self.symbols):
            raise Exception("Invalid index.")

        #sample the index, if required
        if random:
            index = np.random.randint(len(self.symbols))

        #get the target character
        code = self.symbols[index]

        #prepare the result. it must to be unicode
        poison = []

        #calculate the middle of the word
        mid = len(word) // 2

        #create the final message
        poison.append(word[:mid])
        # For now 3, in the midde
        poison.append(code)
        poison.append(code)
        poison.append(code)
        
        poison.append(word[mid:])

        return ''.join(poison)

    ''' define a function that remove Zero-Width SPace (ZWSP) characters '''
    def sanitization(self, sentence):
        #blacklist characters removal
        res = ''.join([c for c in sentence if c not in self.symbols])

        return res

In [ ]:
def replace_word_in_html(html_content, tag_list, target_word, mask, output_path):
    
    soup = BeautifulSoup(open(html_content), 'html.parser')
    
    for tag in tag_list:
        elements= soup.find_all(tag)
        for elem in elements:
            for text_element in elem.find_all(string=True, recursive=True):
                # Only replace the target_word in text nodes
                if target_word in text_element:
                    # Replace target_word with new_word
                    new_text = text_element.replace(target_word, mask)
                    text_element.replace_with(new_text)  # Update the text node with the new text
  
    
    # Save file
    save_html_to_file(str(soup), output_path)


In [ ]:
''' define a function that remove Zero-Width SPace (ZWSP) characters '''
def create_zew_mask1_html(output_path_attak_method, title, body, id):
    path= output_path_attak_method
    zew=ZeroWidthSPaceAttack()
    symbols_len=zew.num_malicius_chars
    
    #set the target word random form body
    target_word=select_target_word(body)
    #select random index
    i_symbol = np.random.randint(symbols_len)

    poisoned_title = title.replace(target_word,zew.mask1(target_word,i_symbol,random=False))
    poisoned_body = body.replace(target_word,zew.mask1(target_word,i_symbol,random=False))
    symbol_string=f"u{ord(zew.symbols[i_symbol]):04X}"
    path_html = os.path.join(path, f"{id}_{symbol_string}_{target_word}.html")

    create_html_with_font(path_html, poisoned_title, poisoned_body,font_path)
    return path_html

def create_zew_mask2_html(output_path_attak_method, title, body, id):
    path= output_path_attak_method
    zew=ZeroWidthSPaceAttack()
    symbols_len=zew.num_malicius_chars
    
    #set the target word random form body
    target_word=select_target_word(body)
    #select random index
    i_symbol = np.random.randint(symbols_len)

    poisoned_title = title.replace(target_word,zew.mask2(target_word,i_symbol,random=False))
    poisoned_body = body.replace(target_word,zew.mask2(target_word,i_symbol,random=False))
    symbol_string=f"u{ord(zew.symbols[i_symbol]):04X}"
    path_html = os.path.join(path, f"{id}_{symbol_string}_{target_word}.html")

    create_html_with_font(path_html, poisoned_title, poisoned_body,font_path)
    return path_html

In [ ]:
#test funtions
if DEBUG:
    title = "i love drugs"
    body = "i love drugs"

    dataset_obf= os.path.join(dataset_path, "Data obfuscation")
    dataset_mask1 = os.path.join(dataset_obf, "Zero-Width_Mask1")
    dataset_mask2 = os.path.join(dataset_obf, "Zero-Width_Mask2")
    if not os.path.exists(dataset_mask1):
        os.makedirs(dataset_mask1)
    if not os.path.exists(dataset_mask2):
        os.makedirs(dataset_mask2)
    print(create_zew_mask1_html(dataset_mask1, title, body, 1))
    print(create_zew_mask2_html(dataset_mask2, title, body, 1))

## Homoglyph Substitution

In [ ]:
confusables = dict()
intentionals = dict()

# Retrieve Unicode Confusable homoglyph characters
conf_resp = requests.get("https://www.unicode.org/Public/security/latest/confusables.txt", stream=True)
for line in conf_resp.iter_lines():
  if len(line):
    line = line.decode('utf-8-sig')
    if line[0] != '#':
      line = line.replace("#*", "#")
      _, line = line.split("#", maxsplit=1)
      if line[3] not in confusables:
        confusables[line[3]] = []
      confusables[line[3]].append(line[7])

# Retrieve Unicode Intentional homoglyph characters
int_resp = requests.get("https://www.unicode.org/Public/security/latest/intentional.txt", stream=True)

for line in int_resp.iter_lines():
  if len(line):
    line = line.decode('utf-8-sig')
    if line[0] != '#':
      line = line.replace("#*", "#")
      _, line = line.split("#", maxsplit=1)
      if line[3] not in intentionals:
        intentionals[line[3]] = []
      intentionals[line[3]].append(line[7])

def replace_text_with_homoglyphs(text, homoglyphs):
  output=""
  for i in text:
      if i in homoglyphs:
          output += random.choice(homoglyphs[i])
      else:
          output += i

  return output

In [ ]:

def create_homoglyph_html(output_path_attak_method, title, body, id):
    path= output_path_attak_method
    #set the target word random form body
    target_word=select_target_word(body)
    obfuscated_word=replace_text_with_homoglyphs(target_word,intentionals)
    while(obfuscated_word==target_word):
       print("Homoglyphs, same obfuscated word generated")
       target_word=select_target_word(body)
       obfuscated_word=replace_text_with_homoglyphs(target_word,intentionals)

    # Replace the target word with a homoglyph
    poisoned_title=title.replace(target_word,obfuscated_word)
    poisoned_body=body.replace(target_word,obfuscated_word)
    path_html = os.path.join(path, f"{id}_{target_word}.html")

    create_html_with_font(path_html, poisoned_title, poisoned_body,font_path)
    return path_html

In [ ]:
if DEBUG:
    homoglyph_path = os.path.join(dataset_obf, "Homoglyph")
    if not os.path.exists(homoglyph_path):
        os.makedirs(homoglyph_path)
    create_homoglyph_html(homoglyph_path, title, body, 1)

## Font poisoning

In [ ]:
def map_letters_in_file(input_file_path, letter_mapping, output_file_path=None):
    """
    Replace letters in HTML text according to a custom mapping.
    
    Args:
        input_file_path (str): Path to input HTML file
        letter_mapping (dict): Dictionary mapping letters to their replacements
        output_file_path (str, optional): Path for output file
    """
    # Set default output path if none provided
    if output_file_path is None:
        base, ext = os.path.splitext(input_file_path)
        output_file_path = f"{base}_mapped{ext}"
    
    # Read input file
    with open(input_file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
    
    def map_letter(char):
        # Only map if character exists in mapping
        return letter_mapping.get(char, char)
    
    # Process all text nodes while preserving HTML structure
    def process_node(element):
        for child in element.children:
            if isinstance(child, str):
                new_text = ''.join(map_letter(c) for c in child)
                child.replace_with(new_text)
            else:
                process_node(child)
    
    # Process all content in body
    if soup.body:
        process_node(soup.body)
    
    # Save the modified HTML
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(str(soup.prettify()))

In [ ]:
def modify_font(html_file, font_file,output_path):
    """
    Modifies the font in the given HTML file to use a custom font.

    Parameters:
        html_file (str): Path to the HTML file to modify.
        font_file (str): Path to the local font file to use (e.g., 'custom-font.ttf').
    
    Returns:
        str: Modified HTML as a string.
    """
    # Load the HTML file
    with open(html_file, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
    
    # Remove existing font-family declarations in <style> blocks
    for style_tag in soup.find_all("style"):
        # Use regex to remove font-family definitions
        style_tag.string = re.sub(r"@font-face\s*{[^}]*}", "", style_tag.string, flags=re.DOTALL)
        style_tag.string = re.sub(r"font-family:[^;]*;", "", style_tag.string if style_tag.string else "")

    # Remove inline font-family declarations
    for tag in soup.find_all(style=True):
        # Split the style attribute and filter out font-family
        styles = tag["style"].split(";")
        filtered_styles = [s for s in styles if not s.strip().startswith("font-family")]
        tag["style"] = ";".join(filtered_styles)

    # Add the new custom font definition in a new <style> tag
    custom_style = f"""
    @font-face {{
        font-family: 'CustomFont';
        src: url('{font_file}') format('truetype');
    }}
    body, h1, h2, h3, p, ul, li {{
        font-family: 'CustomFont', sans-serif !important;
    }}
    """
    new_style_tag = soup.new_tag("style")
    new_style_tag.string = custom_style

    # Insert the new style tag into the <head>
    if soup.head:
        soup.head.append(new_style_tag)
    else:
        head_tag = soup.new_tag("head")
        head_tag.append(new_style_tag)
        soup.insert(0, head_tag)
    save_html_to_file(str(soup), output_path)

In [ ]:
## Font map used to correctly map glyphs to corresponding c
font_map = {
    "a": "z",
    "b": "a", 
    "c": "b",
    "d": "c",
    "e": "d",
    "f": "e",
    "g": "f",
    "h": "g",
    "i": "h",
    "j": "i",
    "k": "j",
    "l": "k",
    "m": "l",
    "n": "m",
    "o": "n",
    "p": "o",
    "q": "p",
    "r": "q",
    "s": "r",
    "t": "s",
    "u": "t",
    "v": "u",
    "w": "v",
    "x": "w",
    "y": "x",
    "z": "y"  # Wrap-around mapping
}
# Apply reverse transformation to text before applying poisoning font
reversed_font_map={val:key for key,val in font_map.items()}


def create_font_poisoning_html(output_path_attak_method, title, body, id):
    font_path_poisoned = os.path.join(pwd, 'PoisonedFont.ttf')
    path = output_path_attak_method
    font_poisoned = os.path.join(pwd, "PoisonedFont.ttf")
    path_html = os.path.join(path, f"{id}_default.html")
    title=title.lower()
    body=body.lower()
    create_html_with_font(path_html, title, body,font_path)
    
    map_letters_in_file(path_html, reversed_font_map, path_html)
    modify_font(path_html, font_poisoned, path_html)

    return path_html

In [ ]:
if DEBUG:
    font_poisoning_path = os.path.join(dataset_obf, "Font_poisoning")
    if not os.path.exists(font_poisoning_path):
        os.makedirs(font_poisoning_path)
    create_font_poisoning_html(font_poisoning_path, title, body, 1)

## Reordering

In [ ]:
# Unicode Bidi override characters
PDF = chr(0x202C)
LRE = chr(0x202A)
RLE = chr(0x202B)
LRO = chr(0x202D)
RLO = chr(0x202E)

PDI = chr(0x2069)
LRI = chr(0x2066)
RLI = chr(0x2067)

# Class to apply Unicode Bidi override characters to obfuscate text
class Swap():
    """Represents swapped elements in a string of text."""
    def __init__(self, one, two):
        self.one = one
        self.two = two
    
    def __repr__(self):
        return f"Swap({self.one}, {self.two})"

    def __eq__(self, other):
        return self.one == other.one and self.two == other.two

    def __hash__(self):
        return hash((self.one, self.two))

def some(*els):
    """Returns the arguments as a tuple with Nones removed."""
    return tuple(filter(None, tuple(els)))

def swaps(chars: str) -> set:
    """Generates all possible swaps for a string."""
    def pairs(chars, pre=(), suf=()):
        orders = set()
        for i in range(len(chars)-1):
            prefix = pre + tuple(chars[:i])
            suffix = suf + tuple(chars[i+2:])
            swap = Swap(chars[i+1], chars[i])
            pair = some(prefix, swap, suffix)
            orders.add(pair)
            orders.update(pairs(suffix, pre=some(prefix, swap)))
            orders.update(pairs(some(prefix, swap), suf=suffix))
        return orders
    return pairs(chars) | {tuple(chars)}

def unswap(el: tuple) -> str:
    """Reverts a tuple of swaps to the original string."""
    if isinstance(el, str):
        return el
    elif isinstance(el, Swap):
        return unswap((el.two, el.one))
    else:
        res = ""
        for e in el:
            res += unswap(e)
        return res

def uniswap(els):
    res = ""
    for el in els:
        if isinstance(el, Swap):
            res += uniswap([LRO, LRI, RLO, LRI, el.one, PDI, LRI, el.two, PDI, PDF, PDI, PDF])
        elif isinstance(el, str):
            res += el
        else:
            for subel in el:
                res += uniswap([subel])
    return res

def strings_to_file(file, string):
  with open(file, 'w') as f:
      for swap in swaps(string):
          uni = uniswap(swap)
          print(uni, file=f)

def print_strings(string):
  for swap in swaps(string):
    uni = uniswap(swap)
    print(uni)

In [ ]:
def create_reordering_attack_html(output_path_attak_method, title, body, id):

    target_word=select_target_word(body)
    swapped_words =swaps(target_word)

    obfuscated_words=[uniswap(swap) for swap in swapped_words]
    #select random obfuscated word
    obfuscated_word = obfuscated_words[np.random.randint(len(obfuscated_words))]

    #manage reorder case where obfuscated word is == target word
    while obfuscated_word == target_word:
        print(f'Reorder attack: Got same words [{target_word},{obfuscated_word}] after reordering. Trying again...')
        obfuscated_word = obfuscated_words[np.random.randint(len(obfuscated_words))]
        
    tag_list=['title','h1','p','li']
    path_html = os.path.join(output_path_attak_method, f"{id}_{target_word}.html")

    create_html_with_font(path_html, title.replace(target_word,obfuscated_word), body.replace(target_word,obfuscated_word),font_path)
    return path_html

In [ ]:
if DEBUG:
    reorder_path = os.path.join(dataset_obf, "Reordering_attacks")
    if not os.path.exists(reorder_path):
        os.makedirs(reorder_path)
    create_reordering_attack_html(reorder_path, title, body, 1)

## Diactricial marks injection

In [ ]:
def has_diacritical_accent(text):
    diacritical_accents = "àèéìòùÀÈÉÌÒÙ"
    return any(char in diacritical_accents for char in text)

In [ ]:
def obfuscate_diacritical(word, times = 10):
    diacritical_accents = "àèéìòùÀÈÉÌÒÙ"
    #chek witch diacritical accent is in the word

    for i,char in enumerate(word):
        if char in diacritical_accents:
            if char in ("à", "è", "ì", "ò", "ù", "À", "È", "É", "Ì", "Ò", "Ù"):
                word = word[:i+1] + '\u0300'*times + word[i+1:]

            else:
                word = word[:i+1] + '\u0301'*times + word[i+1:]
    return word
   
    #replace the accent with the unicode character
diacritical_accents = "àèéìòùÀÈÉÌÒÙ"

def select_target_diacritical_word(text):
    words = re.split(r'\W+', text)  # Split by any non-word characters
    words = [word for word in words if word.isalpha()]  # Keep only words with alphabetic characters
    
    if not words:
        raise ValueError("No valid candidate words found.")
    
    valid_words = [word for word in words if len(word) >= 2]  # Filter words by minimum length of 2 characters
    
    if not valid_words:
        raise ValueError("No valid candidate words found with the required length.")
    
    diacritical_words = [word for word in words if any(char in diacritical_accents for char in word)]
    
    if diacritical_words:
        target_word = random.choice(diacritical_words)
    else:
        #rise an exception
        raise ValueError("No valid candidate diacritial words found.")
    
    return target_word


In [ ]:
def create_diactricial_marks_injection_mask1_html(output_path_attak_method, title, body, id):
    path = output_path_attak_method
    #set the target word random form body
    target_word=select_target_word(body)
    #select random obfuscated word among word with accent
    
    target_word = select_target_diacritical_word(body)
    obfuscated_word = obfuscate_diacritical(target_word)

    

    poisoned_title=title.replace(target_word,obfuscated_word)
    poisoned_body=body.replace(target_word,obfuscated_word)
    
    path_html = os.path.join(path, f"{id}_{target_word}.html")
    create_html_with_roboto(path_html, poisoned_title, poisoned_body)

    return path_html

def create_diactricial_marks_injection_mask2_html(output_path_attak_method, title, body, id):
    path = output_path_attak_method
    #set the target word random form body
    target_word=select_target_word(body)
    #select random obfuscated word among word with accents
        
    target_word = select_target_diacritical_word(body)
    obfuscated_word = obfuscate_diacritical(target_word, 4096**2*2)

    

    poisoned_title=title.replace(target_word,obfuscated_word)
    poisoned_body=body.replace(target_word,obfuscated_word)
    
    path_html = os.path.join(path, f"{id}_{target_word}.html")
    create_html_with_roboto(path_html, poisoned_title, poisoned_body)

    return path_html

In [ ]:
if DEBUG:
    diacritical_mask1_path = os.path.join(dataset_obf, "Diactricial_marks_injection_mask1")
    if not os.path.exists(diacritical_mask1_path):
        os.makedirs(diacritical_mask1_path)
    diacritical_mask2_path = os.path.join(dataset_obf, "Diactricial_marks_injection_mask2")
    if not os.path.exists(diacritical_mask2_path):
        os.makedirs(diacritical_mask2_path)
    title = "i love drùgs"
    body = "i love drùgs and cofè"
    create_diactricial_marks_injection_mask1_html(diacritical_mask1_path, title, body, 1)
    create_diactricial_marks_injection_mask2_html(diacritical_mask2_path, title, body, 2)

## OCR

In [ ]:
import base64

def protect_html_from_ocr(html_content, background_path, output_path):
    """
    Modify the given HTML to include a background image that makes OCR harder.
    
    Args:
        html_content (str): The input HTML content.
        background_path (str): Path to the background image file.

    Returns:
        BeautifulSoup: Modified HTML with OCR protection.
    """
    # Load the background image and convert it to a base64 string
    with open(background_path, "rb") as bg_file:
        bg_base64 = base64.b64encode(bg_file.read()).decode("utf-8")
        bg_data_uri = f"data:image/png;base64,{bg_base64}"
    
    # Parse the HTML content
    soup = BeautifulSoup(open(html_content), "html.parser")
    
    # Inject CSS styles for OCR protection
    style_tag = soup.new_tag("style")
    style_tag.string = f'''
    body {{
        background: url({bg_data_uri}) repeat;
        background-blend-mode: lighten;
        color: black;
        text-shadow: 2px 2px 4px rgba(200, 200, 200, 0.9);
        font-weight: bold;
        position: relative;
    }}
    
    h1, p {{
        position: relative;
        z-index: 10;
    }}
    '''
    
    
    # Add the style tag to the head
    if soup.head:
        soup.head.append(style_tag)
    else:
        head_tag = soup.new_tag("head")
        head_tag.append(style_tag)
        soup.insert(0, head_tag)

    save_html_to_file(str(soup), output_path)

background_path = os.path.join(pwd, "data/images/bgcaptcha.jpeg")
def create_ocr_poisoning_html(output_path_attak_method, title, body, id):
    path = output_path_attak_method
    #set the target word random form body
    path_html = os.path.join(output_path_attak_method, f"{id}_default.html")
    create_html_with_font(path_html, title, body)
    protect_html_from_ocr(path_html, background_path, path_html)
    return path_html

In [ ]:
if DEBUG:
    path_ocr_poisoning = os.path.join(dataset_obf, "OCR-poisoning_injection")
    if not os.path.exists(path_ocr_poisoning):
        os.makedirs(path_ocr_poisoning)
    title = "i love drugs"
    body = "i love drugs"
    create_ocr_poisoning_html(path_ocr_poisoning, title, body, 1)

# Injection words


In [ ]:
words_df=pd.read_csv('unigram_freq.csv')
lowest_freq_words=words_df.sort_values('count',ascending=True).head(2000)['word'].tolist()


In [ ]:
words_current_index = 0

def get_next_word():
    global words_current_index
    word = lowest_freq_words[words_current_index]
    words_current_index = (words_current_index + 1) % len(lowest_freq_words)
    return word


## Transparent

In [ ]:
def add_hidden_text_to_body_bg_color(html_content, tag_name, text_content, background_color, output_path):
    """
    Add an element to the body with text styled to match the background color.

    Parameters:
        html_content (str): The HTML content as a string.
        tag_name (str): The name of the tag to add (e.g., 'h1', 'p').
        text_content (str): The text content of the new element.
        background_color (str): The color of the background and text (e.g., '#ffffff').
        output_path (str): Output path of the file to be saved.

    Returns:
        str: The modified HTML content as a string.
    """
    # Parse the HTML
    soup = BeautifulSoup(open(html_content), 'html.parser')
    
    if soup.body is None:
        raise ValueError("No <body> tag found in the HTML document.")
    
    # Create the new tag
    new_tag = soup.new_tag(tag_name)
    new_tag.string = text_content
    
    # Set the inline style to match text and background color
    new_tag['style'] = f"color: {background_color}; background-color: {background_color};"
    
    # Append the new tag to the body
    soup.body.append(new_tag)

    save_html_to_file(str(soup), output_path)

    

In [ ]:
def add_hidden_text_to_body_opacity(html_content, tag_name, text_content, opacity_level,output_path):
    """
    Add an element to the body with text styled to match the background color.

    Parameters:
        html_content (str): The HTML content as a string.
        tag_name (str): The name of the tag to add (e.g., 'h1', 'p').
        text_content (str): The text content of the new element.
        output_path (str): Output path of the file to be saved.

    Returns:
        str: The modified HTML content as a string.
    """
    # Parse the HTML
    soup = BeautifulSoup(open(html_content), 'html.parser')
    
    if soup.body is None:
        raise ValueError("No <body> tag found in the HTML document.")
    
    # Create the new tag
    new_tag = soup.new_tag(tag_name)
    new_tag.string = text_content
    
    # Set the inline style to match text and background color
    new_tag['style'] = f"opacity: {opacity_level};"
    
    # Append the new tag to the body
    soup.body.append(new_tag)

    save_html_to_file(str(soup), output_path)

In [ ]:
def create_transparent_bg_html(output_path_attak_method, title, body, id):
    #set the target word random form body
    background_color = "transparent"
    tag_name="h1"
    hidden_word = get_next_word()
    path_html = os.path.join(output_path_attak_method, f"{id}_{hidden_word}.html")

    create_html_with_font(path_html, title, body)
    add_hidden_text_to_body_bg_color(path_html, tag_name, hidden_word, background_color, path_html)
    return path_html
def create_transparent_opacity01_html(output_path_attak_method, title, body, id):
    #set the target word random form body
    opacity_level = 0.0001
    tag_name="h1"
    hidden_word = get_next_word()
    path_html = os.path.join(output_path_attak_method, f"{id}_{hidden_word}.html")

    create_html_with_font(path_html, title, body)
    add_hidden_text_to_body_opacity(path_html, tag_name, hidden_word, opacity_level, path_html)
    return path_html

def create_transparent_opacity00_html(output_path_attak_method, title, body, id):
    #set the target word random form body
    opacity_level = 0
    tag_name="h1"
    hidden_word = get_next_word()
    path_html = os.path.join(output_path_attak_method, f"{id}_{hidden_word}.html")

    create_html_with_font(path_html, title, body)
    add_hidden_text_to_body_opacity(path_html, tag_name, hidden_word, opacity_level, path_html)
    return path_html

In [ ]:
if DEBUG:
    path_transparent_text_bg = os.path.join(dataset_injection, "Transparent_Text_injection_background")
    if not os.path.exists(path_transparent_text_bg):
        os.makedirs(path_transparent_text_bg)
    path_transparent_text_opacity = os.path.join(dataset_injection, "Transparent_Text_injection_opacity")
    if not os.path.exists(path_transparent_text_opacity):
        os.makedirs(path_transparent_text_opacity)

    title = "i love drugs"
    body = "i love drugs"
    create_transparent_bg_html(path_transparent_text_bg, title, body, 1)
    create_transparent_opacity00_html(path_transparent_text_opacity, title, body, 1)
    create_transparent_opacity01_html(path_transparent_text_opacity, title, body, 1)

## Out-of-Margin

In [ ]:
def get_random_out_of_bounds_position():
    """
    Generate random out-of-bounds positions based on predefined zones.
    Returns a dictionary containing the 'top' and 'left' CSS values.
    """
    # Define the zones for out-of-bounds positions
    zones = {
        #"top": {"top": f"{random.uniform(-2500, -2000)}px", "left": f"{random.uniform(0, 1000)}px"},
        #"bottom": {"top": f"{random.uniform(2000, 2500)}px", "left": f"{random.uniform(0, 1000)}px"},
        "left": {"top": f"{random.uniform(0, 1000)}px", "left": f"{random.uniform(-2500, -2000)}px"},
        "top_left": {"top": f"{random.uniform(-2500, -2000)}px", "left": f"{random.uniform(-2500, -2000)}px"},
        #"top_right": {"top": f"{random.uniform(-500, -200)}px", "left": f"{random.uniform(2000, 2500)}px"},
        #"bottom_left": {"top": f"{random.uniform(2000,2500)}px", "left": f"{random.uniform(-2500, -2000)}px"},
        "bottom_right": {"top": f"{random.uniform(2000, 2500)}px", "left": f"{random.uniform(-2000, -2500)}px"},
    }
    
    # Randomly choose a zone
    random_zone = random.choice(list(zones.keys()))
    return zones[random_zone]

def add_out_of_bounds_text(html_content, tag_name, text_content, output_path, position="absolute"):
    """
    Add an element with out-of-bounds text using CSS positioning.

    Parameters:
        html_content (str): The HTML content as a string.
        tag_name (str): The name of the tag to add (e.g., 'h1', 'p').
        text_content (str): The text to insert as out-of-bounds content.
        output_path (str): Output path of the file to be saved.
        position (str): The CSS position property (default is 'absolute').
        top (str): The top position (default is '-100px' to move out of view).
        left (str): The left position (default is '-100px' to move out of view).

    Returns:
        str: The modified HTML content.
    """
    soup = BeautifulSoup(open(html_content), 'html.parser')
    
    # Create the new tag (e.g., <div>, <p>, <h1>)
    new_tag = soup.new_tag(tag_name)
    new_tag.string = text_content
    
    # Generate a random out-of-bounds position
    coordinates = get_random_out_of_bounds_position()
    top = coordinates['top']
    left = coordinates['left']

    # Apply CSS to move the text out of bounds
    new_tag['style'] = f"position: {position}; top: {-2500}; left: {left};"
    
    # Append the new tag to the body
    soup.body.append(new_tag)
    
    # Save file 
    save_html_to_file(str(soup), output_path)


def create_out_of_margin_html(output_path_attak_method, title, body, id):
    path = output_path_attak_method
    hidden_word = get_next_word()
    path_html = os.path.join(path, f"{id}_{hidden_word}.html")
    tag_name="p"
    position="absolute"


    create_html_with_font(path_html, title, body, font_path)

    
    add_out_of_bounds_text(path_html,tag_name, hidden_word, path_html, position)

    return path_html

In [ ]:
if DEBUG:
    path_out_of_margin = os.path.join(dataset_injection, "Out_of_margin_injection")
    if not os.path.exists(path_out_of_margin):
        os.makedirs(path_out_of_margin)
    title = "i love drugs"
    body = "i love drugs"
    create_out_of_margin_html(path_out_of_margin, title, body, 1)

## Zero Size

In [ ]:
def inject_zero_size_text_font(html_content, tag_name, text_content, font_size, visibility, output_path):
    """
    Inject text into the body with zero font size to make it invisible to the user.
    
    Parameters:
        html_content (str): The HTML content as a string.
        tag_name (str): The name of the tag to add (e.g., 'h1', 'p').
        text_content (str): The text to inject with zero size.
        output_path (str): Output path of the file to be saved.
        
        
    Returns:
        str: The modified HTML content.
    """
    soup = BeautifulSoup(open(html_content), 'html.parser')
    
    # Create a new tag (e.g., <div>, <p>, <span>)
    new_tag = soup.new_tag(tag_name)
    new_tag.string = text_content
    
    # Apply CSS to set font-size to 0 (invisible)
    new_tag['style'] = f"font-size: {font_size}; visibility: {visibility};"
    
    soup.body.append(new_tag)
    
    # Save file
    save_html_to_file(str(soup), output_path)
    

In [ ]:
def inject_scaled_text(html_content, tag_name, text_content, scale_factor, visibility, output_path):
    """
    Inject text into the body with a CSS transform scale to make it visually hidden.
    
    Parameters:
        html_content (str): The HTML content as a string.
        tag_name (str): The name of the tag to add (e.g., 'h1', 'p').
        text_content (str): The text to inject with scaling.
        scale_factor (float): The scaling factor (e.g., 0.01 for near-invisible text).
        visibility (str): The visibility CSS property value (e.g., 'hidden' or 'visible').
        output_path (str): Output path of the file to be saved.
        
    Returns:
        str: The modified HTML content.
    """
    soup = BeautifulSoup(open(html_content), 'html.parser')
    
    # Create a new tag (e.g., <div>, <p>, <span>)
    new_tag = soup.new_tag(tag_name)
    new_tag.string = text_content
    
    # Apply CSS transform scale to make text nearly invisible
    new_tag['style'] = f"transform: scale({scale_factor}); visibility: {visibility}; display: inline-block;"
    
    soup.body.append(new_tag)
    
    # Save file
    save_html_to_file(str(soup), output_path)

In [ ]:

def create_zero_size_text_font00_html(output_path_attak_method, title, body, id):
    path = output_path_attak_method
    hidden_word = get_next_word()
    path_html = os.path.join(path, f"{id}_{hidden_word}.html")
    tag_name="div"
    font_size= "0"
    visibility="visible"
    create_html_with_font(path_html, title, body, font_path)
    inject_zero_size_text_font(path_html,tag_name, hidden_word, font_size,visibility, path_html)
    return path_html


def create_zero_size_text_font01_html(output_path_attak_method, title, body, id):
    path = output_path_attak_method
    hidden_word = get_next_word()
    path_html = os.path.join(path, f"{id}_{hidden_word}.html")
    tag_name="div"
    font_size= "0.1px"
    visibility="visible"
    create_html_with_font(path_html, title, body, font_path)
    inject_zero_size_text_font(path_html,tag_name, hidden_word, font_size,visibility, path_html)
    return path_html

def create_zero_size_scaled_html(output_path_attak_method, title, body, id):
    path = output_path_attak_method
    hidden_word = get_next_word()
    path_html = os.path.join(path, f"{id}_{hidden_word}.html")
    tag_name="h1"
    scale_factor=0.01
    visibility="visible"
    create_html_with_font(path_html, title, body, font_path)

    inject_scaled_text(path_html,tag_name, hidden_word, scale_factor,visibility, path_html)

    return path_html

def create_zero_size_text_visibilityHidden_html(output_path_attak_method, title, body, id):
    path = output_path_attak_method
    hidden_word = get_next_word()
    path_html = os.path.join(path, f"{id}_{hidden_word}.html")
    tag_name="h1"
    font_size= "18"
    visibility="hidden"
    create_html_with_font(path_html, title, body, font_path)
    inject_zero_size_text_font(path_html,tag_name, hidden_word, font_size,visibility, path_html)
    return path_html

In [ ]:
if DEBUG:
    path_zero_size_text_zerofont = os.path.join(dataset_injection, "Zero-size_Text_injection_font0")
    if not os.path.exists(path_zero_size_text_zerofont):
        os.makedirs(path_zero_size_text_zerofont)
    path_zero_size_text_zerofont01 = os.path.join(dataset_injection, "Zero-size_scaled")
    if not os.path.exists(path_zero_size_text_zerofont01):
        os.makedirs(path_zero_size_text_zerofont01)
    

    create_zero_size_text_font00_html(path_zero_size_text_zerofont, title, body, 1)
    create_zero_size_text_font01_html(path_zero_size_text_zerofont, title, body, 1)
    create_zero_size_scaled_html(path_zero_size_text_zerofont01, title, body, 1)


## Metadata

In [ ]:
def inject_metadata(html_content, metadata, output_path):
    """
    Injects metadata into the <head> section of an HTML document.
    
    Parameters:
    - html_content (str): The HTML content as a file path.
    - metadata (list of dict): A list of metadata dictionaries. Each dict should have 'name' or 'property', and 'content'.
    - output_path (str): The file path to save the modified HTML.
    
    Returns:
    - None: Saves the modified HTML to the output path.
    """
    # Parse the HTML content
    soup = BeautifulSoup(open(html_content), 'html.parser')
    
    # Find or create the <head> section
    head = soup.head
    if not head:
        head = soup.new_tag('head')
        soup.html.insert(0, head)
    
    # Find the style tag if it exists
    style_tag = head.find('style')
    
    # Create all meta tags first
    meta_tags = []
    for meta in metadata:
        meta_tag = soup.new_tag('meta')
        
        # Set attributes in specific order
        if 'name' in meta:
            meta_tag.attrs['name'] = meta['name']
        elif 'property' in meta:
            meta_tag.attrs['property'] = meta['property']
            
        if 'content' in meta:
            meta_tag.attrs['content'] = meta['content']
            
        meta_tags.append(meta_tag)
    
    # Insert meta tags in appropriate location
    if style_tag:
        # Insert before style tag
        for meta_tag in meta_tags:
            style_tag.insert_before(meta_tag)
    else:
        # If no style tag, append to head
        for meta_tag in meta_tags:
            head.append(meta_tag)
    
    save_html_to_file(str(soup), output_path)

In [ ]:
def create_metadata_hidden_text_html(output_path_attak_method, title, body, id):
    path = output_path_attak_method
    hidden_word = get_next_word()
    path_html = os.path.join(path, f"{id}_{hidden_word}.html")

    create_html_with_font(path_html, title, body, font_path)
    metadata_to_add = [
        {"name": "description", "content": f"{hidden_word}"},
        {"property": "og:summary", "content": f"{hidden_word}"}
    ]

    inject_metadata(path_html, metadata_to_add, path_html)


    return path_html

In [ ]:
if DEBUG:
    path_metadata_hidden_text = os.path.join(dataset_injection, "Metadata_Hidden_Text_injection")
    if not os.path.exists(path_metadata_hidden_text):
        os.makedirs(path_metadata_hidden_text)

    create_metadata_hidden_text_html(path_metadata_hidden_text, title, body, 1)

## Decive Element

In [ ]:
from bs4 import BeautifulSoup
import os
import random

def inject_hidden_text_behind_image(path_html, tag_name, image_selector, hidden_text, output_path, path_image):
    """
    Injects hidden text behind the specified image in the HTML content.

    Parameters:
    - path_html (str): The path to the HTML file.
    - image_selector (str): The selector (e.g., id, class, or tag) for the target image.
    - hidden_text (str): The text to inject behind the image.
    - output_path (str): Output path of the file to be saved.
    - path_image (str): Path to the image file to be used.

    Returns:
    - str: The modified HTML content.
    """
    # Parse the HTML
    with open(path_html, 'r') as file:
        soup = BeautifulSoup(file, 'html.parser')
    
    # Find the image element using the selector
    image = soup.select_one(image_selector)
    
    # If the image is not found, create and insert it
    if not image:
        image = soup.new_tag('img', src=path_image, id=image_selector.strip('#'))
        soup.body.append(image)
    else:
        # Update the image source to use the provided path_image
        image['src'] = path_image
    
    # Create a hidden text element
    hidden_text_element = soup.new_tag(
        tag_name, 
        style=(
            "position: absolute; "
            "z-index: -1; "  # Ensures it's behind the image
            "width: 100%; "  # Makes the text span the entire width of its parent
            "left: 0; "      # Align to the left edge of the parent
            "transform: translateY(-50%); "  # Center vertically relative to its position
            "text-align: center; "  # Ensure the text is invisible
        )
    )
    hidden_text_element.string = hidden_text
    
    # Insert the hidden text behind the image
    image.insert_before(hidden_text_element)

    # Save the modified HTML content to the output path
    with open(output_path, 'w') as file:
        file.write(str(soup))

# Example usage in create_decived_html function
def create_decived_html(path, title, body, id):
    hidden_word = get_next_word()
    path_html = os.path.join(path, f"{id}_{hidden_word}.html")

    tag_name = "h2"
    image_selector = "#target-image"  # This should match the ID of the target image in your HTML

    create_html_with_font(path_html, title, body, font_path)
    image_path = os.path.join(pwd, "data/images/logo.png")
    inject_hidden_text_behind_image(path_html, tag_name, image_selector, hidden_word, path_html, image_path)

In [ ]:
if DEBUG:
    path_text_under_image = os.path.join(dataset_injection, "Decived_injection")
    if not os.path.exists(path_text_under_image):
        os.makedirs(path_text_under_image)

    create_decived_html(path_text_under_image, title, body, 1)


# CREATION DATASET

In [ ]:
#restart dataset iterator
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Grocery_and_Gourmet_Food", streaming=True, trust_remote_code=True)
dataset_iterator = iter(dataset["full"])

In [ ]:
attack_methods_obfuscation = {
    "Zero-Width_mask1": create_zew_mask1_html,
    "Zero-Width_mask2": create_zew_mask2_html,
    "Homoglyph_default": create_homoglyph_html,
    "Font-poisoning_default": create_font_poisoning_html,
    "Reordering-attack_default": create_reordering_attack_html,
    "Diacritical-injection_mask1": create_diactricial_marks_injection_mask1_html,
    #"Diactricial marks injection-mask2": create_diactricial_marks_injection_mask2_html, #DDOS
    "OCR-poisoning_default": create_ocr_poisoning_html
}

attack_methods_injection = {
    "Transparent-Text-injection_background-color": create_transparent_bg_html,
    "Transparent-Text-injection_opacity00": create_transparent_opacity00_html,
    "Transparent-Text-injection_opacity01": create_transparent_opacity00_html,
    "Out-of-bound-injection_default": create_out_of_margin_html,
    "Zero-size-injection_font00": create_zero_size_text_font00_html,
    "Zero-size-injection_font01": create_zero_size_text_font01_html,
    "Zero-size-injection_scaling": create_zero_size_scaled_html,
    "Zero-size-injection_visibilityHidden": create_zero_size_text_visibilityHidden_html,
    "Deceived-element-injection_default": create_decived_html,
    "Metadata-injection_default": create_metadata_hidden_text_html
}

super_class_map = {
    "Data obfuscation": attack_methods_obfuscation,
    "Poisoned text injection": attack_methods_injection
}

In [ ]:
RECREATE = True

In [ ]:
import os
import glob
import shutil

# Ottieni il percorso corrente
pwd = os.path.dirname(os.getcwd())

# Definisci il percorso della cartella dataset
dataset_path = os.path.join(pwd, "data", "dataset", "HTML")
print(dataset_path)
if RECREATE:
    # Crea la cartella se non esiste
    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path)
    else:
        # Elimina tutti i file e le directory nella cartella dataset
        files = glob.glob(os.path.join(dataset_path, '*'))
        for f in files:
            if os.path.isfile(f):
                os.remove(f)
            elif os.path.isdir(f):
                shutil.rmtree(f)

In [ ]:
def extract_sanitized(dataset_iterator):
    
    data_element = next(dataset_iterator)
    title = data_element["title"]
    text = data_element["text"]
    
    if not re.search(r'\w+', text, re.IGNORECASE):
        raise ValueError("No valid candidate words found. No words in the body")
    return title, text

In [ ]:
import sys
num_files = 100
count = 0
#crete the folders folder_names if not existing
for super_class in super_class_map:
    if not os.path.exists(os.path.join(dataset_path, super_class)):
        os.makedirs(os.path.join(dataset_path, super_class))
    else :
        print("folder already exists")
        # #remove all files in the folder
        shutil.rmtree(os.path.join(dataset_path, super_class))
        os.makedirs(os.path.join(dataset_path, super_class))
    for sub_class in super_class_map[super_class]:
        if not os.path.exists(os.path.join(dataset_path, super_class, sub_class)):
            os.makedirs(os.path.join(dataset_path, super_class, sub_class))
            print(os.path.join(dataset_path, super_class, sub_class))
            path_pdf = os.path.join(dataset_path, super_class, sub_class)
        
        #ocr
        if (sub_class ==  "OCR-poisoning_default") :
            ocr_mapping_df_path="../ocr_mapping.csv"
            #if file does not exist, create new df
            if not os.path.exists(ocr_mapping_df_path):
                ocr_df=pd.DataFrame(columns=["file", "full_file_path","title",'text','joint_text'])
                ocr_df.to_csv(ocr_mapping_df_path, index=False)
            else:
                ocr_df=pd.read_csv(ocr_mapping_df_path)
    
            for i in range(num_files):    
                while True:  # Loop to retry if an error occurs
                    try:

                        title, text = extract_sanitized(dataset_iterator)

                        joint_text = title + "\n\n" + text
                        

                        creation = super_class_map[super_class][sub_class]
                        path_pdf_file=creation(path_pdf, title, text, i)
                        print(path_pdf_file)

                        file_name=os.path.basename(path_pdf_file)

                        ocr_df.loc[len(ocr_df)] = {
                            "file": file_name,
                            "full_file_path": path_pdf_file,
                            "title": title,
                            "text": text,
                            "joint_text": joint_text
                        }
                        break  # Exit the loop if successful
                    except StopIteration:
                        print("No more elements in the dataset.")
                        break
                    except Exception as e:
                        print(f"Error occurred: {e}. Retrying with the next dataset element..., cout error {count}")
                        print(path_pdf)
                        print (title)
                        print (text)
                        count += 1
                        continue  # Retry with the next dataset element
            ocr_df.to_csv(ocr_mapping_df_path, index=False)

        elif (sub_class=='Font-poisoning_default'):
            font_poisoning_mapping_df_path="../font_poisoning_mapping.csv"
            #if file does not exist, create new df
            if not os.path.exists(font_poisoning_mapping_df_path):
                font_poisoning_df=pd.DataFrame(columns=["file","full_file_path", "title",'text','joint_text'])
                font_poisoning_df.to_csv(font_poisoning_mapping_df_path, index=False)
            else:
                font_poisoning_df=pd.read_csv(font_poisoning_mapping_df_path)
    
            for i in range(num_files):    
                while True:  # Loop to retry if an error occurs
                    try:
                        title, text = extract_sanitized(dataset_iterator)


                        joint_text = title + "\n\n" + text
                        
                        creation = super_class_map[super_class][sub_class]
                        path_pdf_file=creation(path_pdf, title, text, i)
                        print(path_pdf_file)

                        file_name=os.path.basename(path_pdf_file)
                        font_poisoning_df.loc[len(font_poisoning_df)] = {
                            "file": file_name,
                            "full_file_path": path_pdf_file,
                            "title": title,
                            "text": text,
                            "joint_text": joint_text
                        }

                        break  # Exit the loop if successful
                    except StopIteration:
                        print("No more elements in the dataset.")
                        break
                    except Exception as e:
                        print(f"Error occurred: {e}. Retrying with the next dataset element..., cout error {count}")
                        print(path_pdf)
                        print (title)
                        print (text)
                        count += 1
                        continue  # Retry with the next dataset element
            font_poisoning_df.to_csv(font_poisoning_mapping_df_path,index=False)
        
        elif (sub_class ==  "Diacritical-injection_mask1" or sub_class ==  "Diacritical-injection_mask2") :
            for i in range(num_files):    
                
                # Iterate until a title with a diacritical accent is found
                found = False
                while not found:
                    try:
                        title, text = extract_sanitized(dataset_iterator)

                        
                        if has_diacritical_accent(title):                       
                            # Create the PDF
                            print(path_pdf)
                            creation = super_class_map[super_class][sub_class]
                            print(creation(path_pdf, title, text, i))
                            found = True
                            
                    except StopIteration:
                        print("No more elements in the dataset.")
                        break  # Exit the loop if no more elements are available
                    except Exception as e:
                        print(f"Error occurred: {e}. Retrying with the next dataset element...")
                        continue  # Retry with the next element if an error occurs

    
        else :
            for i in range(num_files):    
                while True:  # Loop to retry if an error occurs
                    try:
                        title, text = extract_sanitized(dataset_iterator)
                        
                        creation = super_class_map[super_class][sub_class]
                        print(creation(path_pdf, title, text, i))
                        break  # Exit the loop if successful
                    except StopIteration:
                        print("No more elements in the dataset.")
                        break
                    except Exception as e:
                        print(f"Error occurred: {e}. Retrying with the next dataset element..., cout error {count}")
                        #print(path_pdf)
                        #print (title)
                        #print (text)
                        count += 1
                        continue  # Retry with the next dataset element
